In [1]:
import requests 
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import os

In [15]:
import chromedriver_autoinstaller


chromedriver_autoinstaller.install()

'c:\\Users\\Bened\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\chromedriver_autoinstaller\\107\\chromedriver.exe'

In [230]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import math



# Make sure chrome driver is in your path,
# full download totorial on official python selenium website.

# Start broswer with sudoku.com
driver = webdriver.Chrome()


In [185]:
commissioners = {
    '184': 'Phil Hogan',
    '881': 'Ursula von der Leyen',
    '981': 'Josep Borrell Fontelles',
    '991': 'Thierry Breton',
    '990': 'Helena Dalli',
    '166': 'Valdis Dombrovskis',
    '987': 'Elisa Ferreira',
    '761': 'Mariya Gabriel',
    '985': 'Paolo Gentiloni',
    '180': 'Johannes Hahn',
    '992': 'Ylva Johansson',
    '186': 'Věra Jourová',
    '988': 'Stella Kyriakides',
    '993': 'Janez Lenarčič',
    '1201': 'Mairead McGuinness',
    '989': 'Didier Reynders',
    '983': 'Margaritis Schinas',
    '181': 'Margrethe Vestager',
    '984': 'Nicolas Schmit',
    '986': 'Janusz Wojciechowski',
    '183': 'Maroš Šefčovič',
    '995': 'Kadri Simson',
    '996': 'Virginijus Sinkevičius',
    '982': 'Dubravka Šuica',
    '162': 'Frans Timmermans',
    '994': 'Jutta Urpilainen',
    '997': 'Adina Vălean',
    '998': 'Olivér Várhelyi'
}

In [186]:
def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

In [187]:
roundup(2067)

2070

In [197]:
links = []
for key in commissioners:
    print(f"Next Commissioner: {commissioners[key]}")

    driver.get(f"https://ec.europa.eu/commission/presscorner/advancedsearch/en?keywords=&dotyp=4&parea=&pareaType=&datepickerbefore=&datebefore=&commissioner={key}&datepickerafter=&dateafter=&pagenumber=")

    time.sleep(2.5)
    
    n_results = driver.find_element(By.XPATH, '/html/body/app-root/app-advancedsearch/main/div/div[2]/section/div[1]/span').text.replace("(", "").replace(")", "")

    n_pages = int(roundup(int(n_results))/10)

    print(f"Number of Pages to Parse: {n_pages}")

    for i in range(n_pages):
        i = i+1

        driver.get(f"https://ec.europa.eu/commission/presscorner/advancedsearch/en?keywords=&dotyp=4&parea=&pareaType=&datepickerbefore=&datebefore=&commissioner={key}&datepickerafter=&dateafter=&pagenumber=" + str(i))

        time.sleep(2.5)

        speeches = driver.find_elements(By.CLASS_NAME, "ecl-list-item__link")

        links = links + [{'speaker': commissioners[key], 'speaker_id': key, 'link': elem.get_attribute('href')} for elem in speeches]





Next Commissioner: Phil Hogan
Number of Pages to Parse: 27
Next Commissioner: Ursula von der Leyen
Number of Pages to Parse: 27
Next Commissioner: Josep Borrell Fontelles
Number of Pages to Parse: 1
Next Commissioner: Thierry Breton
Number of Pages to Parse: 3
Next Commissioner: Helena Dalli
Number of Pages to Parse: 1
Next Commissioner: Valdis Dombrovskis
Number of Pages to Parse: 49
Next Commissioner: Elisa Ferreira
Number of Pages to Parse: 1
Next Commissioner: Mariya Gabriel
Number of Pages to Parse: 21
Next Commissioner: Paolo Gentiloni
Number of Pages to Parse: 8
Next Commissioner: Johannes Hahn
Number of Pages to Parse: 5
Next Commissioner: Ylva Johansson
Number of Pages to Parse: 1
Next Commissioner: Věra Jourová
Number of Pages to Parse: 22
Next Commissioner: Stella Kyriakides
Number of Pages to Parse: 4
Next Commissioner: Janez Lenarčič
Number of Pages to Parse: 1
Next Commissioner: Mairead McGuinness
Number of Pages to Parse: 1
Next Commissioner: Didier Reynders
Number of Pa

In [199]:
len(links)

2150

In [211]:
from tqdm import tqdm

In [262]:
out = []
for link in tqdm(links):

    try:
        driver.get(link['link'])
    except:
        continue
    
    time.sleep(3)

    try:
        el = driver.find_element(By.CLASS_NAME, "ecl-paragraph")
        speech = el.text
        speech = speech.replace('\n', ' ')
        speech = speech.replace('(Check against delivery)', '').strip()
    except:
        speech = "None"

    try:
        title = driver.find_element(By.CLASS_NAME, "ecl-page-header__title").find_element(By.CSS_SELECTOR, 'h1').text

        meta = driver.find_element(By.CLASS_NAME, "ecl-meta--header").find_elements(By.CLASS_NAME, 'ecl-meta__item')

        meta_tags = [p.text for p in meta]
    except:
        title = "None"
        meta = "None"
        meta_tags = "None"

    try:
        speech_id = driver.find_element(By.CLASS_NAME, "ecpr-reference__display").text
    except:
        speech_id = "None"

    try:
        date = [i for i in meta_tags if "20" in i][0]
    except:
        date = "None"

    try:
        year = [i for i in meta_tags if "20" in i][0].split(" ")[-1]
    except:
        year = "None"


    item = {
        'speech_id': speech_id,
        'speaker': link['speaker'],
        'speaker_id': link['speaker_id'],
        'date': date,
        'year': year,
        'title': title,
        'meta_tags': '|'.join(meta_tags),
        'link':link['link'],
        'speech': speech,
    }

    out.append(item)



100%|██████████| 2150/2150 [1:59:34<00:00,  3.34s/it]  


In [263]:
out[1]

{'speech_id': 'SPEECH/19/7322',
 'speaker': 'Phil Hogan',
 'speaker_id': '184',
 'date': '14 October 2019',
 'year': '2019',
 'title': 'Intervention for the Plenary on SOFI Launch (Rome, 14/10/2019)',
 'meta_tags': 'Speech|14 October 2019|Brussels',
 'link': 'https://ec.europa.eu/commission/presscorner/detail/en/speech_19_7322',
 'speech': "Distinguished Guests, Ladies and Gentlemen, From the very start of my mandate in the European Commission, I have put sustainable agriculture, food security and nutrition at the heart of our work on international cooperation and development. Because, it is absolutely clear that solving the hunger question is decisive for our work on all 17 Sustainable Development Goals. However, this year's SOFI report is a stark reminder that we are not making progress and that we have to change course. There are too many people still going hungry. Too many people without access to safe and nutritious food. And too many stunted children. Having adequate food for all

In [236]:
try:
    driver.get('https://ec.europa.eu/commission/presscorner/detail/en/speech_19_7314')
except:
    pass

time.sleep(3)

el = driver.find_element(By.CLASS_NAME, "ecl-paragraph")
paragraphs = el.text


paragraphs

"I am delighted to be here in Addis Ababa alongside Chairperson Faki to sign the European Union's contribution of 800 million euros to the African Peace Facility for the period 2019-2020.\nToday's signature brings the total European Union support to African efforts for peace and security to 3.4 billion euros. In signing this agreement today, we commit substantial funding and contribute to the strategic partnership that Europe and Africa are building together in all areas, including peace and security.\nIt is a partnership that we can be proud of. It shows our shared understanding of the need to address common threats and to secure the best conditions for growth and development in Africa.\nBecause it goes without saying that, wherever in the world we may live, peace and security offer the basic climate in which economic, social and human development can thrive.\nBut let me be clear: Africa is in the driving seat; we want to support our African partners in delivering African solutions to

In [227]:
out[1000]

IndexError: list index out of range

In [281]:
links_df = pd.DataFrame.from_dict(links)
links_df

,speaker,speaker_id,link
0,Phil Hogan,184,https://ec.europa.eu/commission/presscorner/de...
1,Phil Hogan,184,https://ec.europa.eu/commission/presscorner/de...
2,Phil Hogan,184,https://ec.europa.eu/commission/presscorner/de...
3,Phil Hogan,184,https://ec.europa.eu/commission/presscorner/de...
4,Phil Hogan,184,https://ec.europa.eu/commission/presscorner/de...
...,...,...,...
2145,Adina Vălean,997,https://ec.europa.eu/commission/presscorner/de...
2146,Adina Vălean,997,https://ec.europa.eu/commission/presscorner/de...
2147,Adina Vălean,997,https://ec.europa.eu/commission/presscorner/de...
2148,Olivér Várhelyi,998,https://ec.europa.eu/commission/presscorner/de...


In [264]:
speeches_df = pd.DataFrame.from_dict(out)
speeches_df

,speech_id,speaker,speaker_id,date,year,title,meta_tags,link,speech
0,SPEECH/19/7325,Phil Hogan,184,14 October 2019,2019,Side Event of the Global Network Against Food ...,Speech|14 October 2019|Brussels,https://ec.europa.eu/commission/presscorner/de...,"Dear David, Dear Dongyu, Distinguished Guests,..."
1,SPEECH/19/7322,Phil Hogan,184,14 October 2019,2019,Intervention for the Plenary on SOFI Launch (R...,Speech|14 October 2019|Brussels,https://ec.europa.eu/commission/presscorner/de...,"Distinguished Guests, Ladies and Gentlemen, Fr..."
2,SPEECH/19/7324,Phil Hogan,184,14 October 2019,2019,"Signature 9 million euros contract (Rome, 14/1...",Speech|14 October 2019|Brussels,https://ec.europa.eu/commission/presscorner/de...,"Ladies and Gentlemen, Dear colleagues, Dear Qu..."
3,SPEECH/19/7277,Phil Hogan,184,30 September 2019,2019,Trade For Her – Empowering women through inter...,Speech|30 September 2019|Brussels,https://ec.europa.eu/commission/presscorner/de...,"Ladies and Gentlemen, Dear Cecilia, First, let..."
4,SPEECH/19/7309,Phil Hogan,184,26 September 2019,2019,Integrated National Financing Frameworks – a t...,Speech|26 September 2019|Brussels,https://ec.europa.eu/commission/presscorner/de...,"Your Excellencies, Distinguished Guests, Ladie..."
...,...,...,...,...,...,...,...,...,...
2145,SPEECH/22/6767,Adina Vălean,997,10 November 2022,2022,Press conference on the Security and Defence P...,Speech|10 November 2022|Brussels,https://ec.europa.eu/commission/presscorner/de...,Speech by Executive Vice-President Margrethe V...
2146,SPEECH/21/3707,Adina Vălean,997,15 July 2021,2021,Opening remarks by Executive Vice-President Ti...,Speech|15 July 2021|Brussels,https://ec.europa.eu/commission/presscorner/de...,Opening remarks by Executive Vice-President Ti...
2147,SPEECH/20/2370,Adina Vălean,997,9 December 2020,2020,Remarks by Executive Vice President Timmermans...,Speech|9 December 2020|Brussels,https://ec.europa.eu/commission/presscorner/de...,"Executive Vice-President Timmermans, responsib..."
2148,SPEECH/22/6144,Olivér Várhelyi,998,12 October 2022,2022,Opening remarks by Commissioner Olivér Várhely...,Speech|12 October 2022|Brussels,https://ec.europa.eu/commission/presscorner/de...,"Good afternoon, ladies and gentlemen. Welcome ..."


In [282]:
links_df.to_csv("links.csv", sep=";")

In [280]:
speeches_df.to_csv("speeches.csv", sep=";")

In [268]:
test = []
for link in tqdm(links):

    try:
        #driver.get(link['link'])
        driver.get('https://ec.europa.eu/commission/presscorner/detail/en/speech_15_6858')
    except:
        continue
    
    time.sleep(3)

    try:
        el = driver.find_element(By.CLASS_NAME, "ecl-paragraph")
        speech = el.text
        speech = speech.replace('\n', ' ')
        speech = speech.replace('(Check against delivery)', '').strip()
    except:
        speech = "None"

    try:
        title = driver.find_element(By.CLASS_NAME, "ecl-page-header__title").find_element(By.CSS_SELECTOR, 'h1').text

        meta = driver.find_element(By.CLASS_NAME, "ecl-meta--header").find_elements(By.CLASS_NAME, 'ecl-meta__item')

        meta_tags = [p.text for p in meta]
    except:
        title = "None"
        meta = "None"
        meta_tags = "None"

    try:
        speech_id = driver.find_element(By.CLASS_NAME, "ecpr-reference__display").text
    except:
        speech_id = "None"

    try:
        date = [i for i in meta_tags if "20" in i][0]
    except:
        date = "None"

    try:
        year = [i for i in meta_tags if "20" in i][0].split(" ")[-1]
    except:
        year = "None"


    item = {
        'speech_id': speech_id,
        'speaker': link['speaker'],
        'speaker_id': link['speaker_id'],
        'date': date,
        'year': year,
        'title': title,
        'meta_tags': '|'.join(meta_tags),
        'link':link['link'],
        'speech': speech,
    }

    test.append(item)

    break

  0%|          | 0/2150 [00:03<?, ?it/s]


In [279]:
speeches_df[speeches_df['speech_id'] == "SPEECH/19/7134"]['speech'][24]

"President Nyusi, Your Excellencies, Ladies and Gentlemen, Let me first express my most sincere condolences to all those who have been affected by cyclones Idai and Kenneth. Too many lives lost; Too many people going hungry; Too many homes and buildings destroyed or damaged. As one of Mozambique's long-standing partners, the European Union is – and will continue to be – by your side in these difficult times. Humanitarian efforts have been very significant and helped with searches, evacuations, food distribution, and shelter The cyclones also triggered one of the biggest EU civil protection operations to date, with over 170 doctors and nurses from EU Member States deployed in Mozambique and the wider region. However, today's conference marks the beginning of a new phase: The one of recovery and reconstruction. So today, let me stress the European Union's unflinching solidarity and support to Mozambique, as it begins this long task. I am pleased to announce an additional 200 million euro

In [269]:
test

[{'speech_id': 'SPEECH/15/6858',
  'speaker': 'Phil Hogan',
  'speaker_id': '184',
  'date': '23 June 2015',
  'year': '2015',
  'title': "'A new start for Europe: Opening up to an ERA of Innovation'",
  'meta_tags': 'Speech|23 June 2015|Brussels',
  'link': 'https://ec.europa.eu/commission/presscorner/detail/en/speech_19_7325',
  'speech': 'ERA conference First of all, thank you for the opportunity to speak here today. As the Commission\'s Spring Economic Forecast showed, we are currently witnessing cautious signs of an economic upturn across Europe. I think we can all agree that Europe will only be able to make a new start by turning this upturn into a lasting and solid economic recovery. This is why kick-starting economic growth and job recovery has been the number one priority of this Commission since it took office. Through the European Semester 2015, the Commission presented an ‘integrated approach\' on how to achieve this goal built on three pillars: Boosting investment through 